# **Reti Neurali** 

## **Esercizio 1: Download e pre-processamento dei dati.**

Scaricare e pre-processare i dati per il successivo addestramento del modello. 

Il dataset che utilizzeremo sarà CIFAR10, scaricabile dalla libreria `tensorflow.keras.datasets`

In [7]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

# Download dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train = y_train.ravel()
y_test = y_test.ravel()

# Stampare le shape dei dati
print(x_train.shape[0], y_train.shape, x_test.shape, y_test.shape)

50000 (50000,) (10000, 32, 32, 3) (10000,)


In [2]:
from sklearn.preprocessing import StandardScaler

# Pre-processamento dei dati

# svolgimento...
x_train=x_train.reshape(x_train.shape[0], -1)
x_test=x_test.reshape(x_test.shape[0], -1)

print(x_train.shape, x_test.shape)

SC=StandardScaler()
x_train_std=SC.fit_transform(x_train)
x_test_std=SC.transform(x_test)

(50000, 3072) (10000, 3072)


## **Esercizio 2: Creare modello MLP**

Per creare il modello MLP utilizziamo l' oggetto `MLPClassifier` dal modulo `sklearn.neural_networks`. Questo è un oggetto molto complesso che prevede la possibilità di specificare tanti parametri, permettendoci una personalizzazione molto dettagliata. Vediamo di seguito gli argomenti principali:

- `hidden_layer_sizes`: rappresenta la struttura dell' MLP, sotto forma di una tupla. La tupla deve essere composta da numeri interi, ogni numero indica il numero di neuroni presenti nel rispettivo layer.

Esempio: 

`hidden_layer_sizes` = `(100)`

creerà un solo layer con 100 neuroni

`hidden_layer_sizes` = `(100, 50)`

creerà due layer, il primo con 100 neuroni, il secondo invece con 50.

- `max_iter`: massimo numero di iterazioni per raggiungere la convergenza. 

- `activation`: indica quale funzione di attivazione utilizzare, valori possibili sono `'relu'`, `'logistic'`, `'tanh'` and `'identity'`.

- `solver`: indica quale algoritmo di ottimizzazione utilizzare, valori possibili sono `'adam'`, `'sgd'` and `'lbfgs'`.

- `learning_rate_init`: valore iniziale del learning rate.

- `verbose`: valore booleano che, se impostato su `True`, stampa l' output di ogni iterazione di training. Molto utile per monitorare il training.

- `random_state`: fissa il seed della randomizzazione.


Per iniziare creiamo un MLP molto basilare, alleniamolo e testiamone le performance. Come parametri utilizzeremo:

- `hidden_layer_sizes` = `(100)`

- `max_iter` = `20`

- `random_state` = `42`

In [3]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Creare MLP 

# svolgimento...
mlp=MLPClassifier(hidden_layer_sizes = (100,50,50,50,25,25,25), max_iter = 20, random_state = 42)


# Allenare MLP

# svolgimento...
mlp.fit(x_train_std, y_train)
y_predicted=mlp.predict(x_test_std)

# Valutare MLP

# svolgimento...
accuracy_test=accuracy_score(y_test, y_predicted)


# Stampare l' accuratezza

# svolgimento...
print(accuracy_test)

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


0.5162


## **Esercizio 2.1: Aumentiamo i parametri del nostro modello**

Proviamo adesso ad aumentare i dettagli del nostro modello, modificando o aggiungendo i parametri sopra specificati. 

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Creare MLP con più strati e altre specifiche

# svolgimento...
mlp=MLPClassifier(hidden_layer_sizes = (100,50,50,50,25,25,25) , max_iter = 20, random_state = 42, activation='logistic', solver='lbfgs')

mlp.fit(x_train_std, y_train)
y_predicted=mlp.predict(x_test_std)

accuracy_test=accuracy_score(y_test, y_predicted)

print(accuracy_test)

0.1


## **Esercizio 3: Implementare manualmente l' algoritmo di early stopping.**

L' algoritmo di early stopping ci permette di terminare anticipatamente l' allenamento di un modello nel caso in cui questo raggiunga la convergenza. Supponiamo infatti che il nostro modello raggiunga un certo livello di accuratezza e che non riesca a migliorare oltre quel livello. Questo significa che il modello, da quel momento in poi, non sta più apprendendo nuove informazioni, per cui le successive iterazioni sono superflue, ed inoltre rischiano di essere dannose, spingendo il modello verso l' overfitting. 

L' early stopping verifica ad ogni iterazione che l' accuratezza del modello sia incrementata di una certa tolleranza. Se questa tolleranza non viene superata per un certo numero di epoche, allora possiamo decidere di stoppare l' allenamento in quanto il modello ha raggiunto la convergenza.

**N.B: per applicare early stopping è necessario specificare i seguenti parametri dell' MLP:**

- `warm_start`=`True` in modo che il training proceda dallo stato attuale del modello e non dall' inizializzazione.

- `max_iter`=`1` in modo che il modello venga allenato per una sola epoca. Per l' early stopping infatti dovremo gestire manualmente il numero di iterazioni.

In [8]:
import numpy as np
from sklearn.neural_network import MLPClassifier 
from sklearn.metrics import accuracy_score 


n_total_epochs = 100  
patience = 10         
tolerance = 1e-4      

best_test_accuracy = 0.0
epochs_without_improvement = 0

validation_size=0.2
x_val=x_train_std[:int(validation_size*len(x_train_std))]
y_val=y_train[:int(validation_size*len(x_train_std))]

x_train=x_train_std[int(validation_size*len(x_train_std)):]
y_train=y_train[int(validation_size*len(x_train_std)):]

print(x_train.shape,x_val.shape, y_train.shape, y_val.shape)

mlp=MLPClassifier(hidden_layer_sizes = (100,100,50,50,25,25,25),  max_iter = 1, random_state = 42, warm_start=True, verbose=True)

for i in range(0, n_total_epochs):
    mlp.fit(x_train, y_train)
    y_predicted=mlp.predict(x_val)
    accuracy_val=accuracy_score(y_val, y_predicted)
    if(accuracy_val-best_test_accuracy>tolerance):
        epochs_without_improvement=0
        best_test_accuracy=accuracy_val 
    else:
        epochs_without_improvement+=1
    if(epochs_without_improvement==patience):
        break

(40000, 3072) (10000, 3072) (40000,) (10000,)
Iteration 1, loss = 1.85370760


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 2, loss = 1.62540576
Iteration 3, loss = 1.51824089
Iteration 4, loss = 1.43872934
Iteration 5, loss = 1.37383048
Iteration 6, loss = 1.31698496
Iteration 7, loss = 1.26368998
Iteration 8, loss = 1.21912949
Iteration 9, loss = 1.17688997
Iteration 10, loss = 1.14417898
Iteration 11, loss = 1.10851941
Iteration 12, loss = 1.07804479
Iteration 13, loss = 1.05416696
Iteration 14, loss = 1.03252151
Iteration 15, loss = 1.01710360
Iteration 16, loss = 0.99544901
Iteration 17, loss = 0.97167743
Iteration 18, loss = 0.95095925
Iteration 19, loss = 0.93170275
Iteration 20, loss = 0.90829877


In [9]:
print(best_test_accuracy)

0.4865
